### It turns out that Epoch = 3, Batch Size = 8 , Learning Rate = 5e-5 performs well (without overfitting)

In [ ]:
!pip install datasets transformers
!pip install emoji

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd

from datasets import Dataset
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import DefaultDataCollator

In [ ]:
##BERT_MODEL = "bert-base-uncased"
BERT_MODEL = "vinai/bertweet-base"
#BERT_MODEL = "vinai/bertweet-large"
NUM_EPOCHS = 25
BATCH_SIZE = 32
DROPOUT = 0.1

## Load Data into Pandas DF

In [ ]:

data_train = pd.read_csv("/content/drive/My Drive/trunc_data_train.csv")  
data_train.fillna(" ", inplace=True)
data_dev = pd.read_csv("/content/drive/My Drive/trunc_data_dev.csv")
data_dev.fillna(" ", inplace=True)


In [ ]:
dataset_train = Dataset.from_pandas(data_train)
dataset_train

Dataset({
    features: ['Unnamed: 0', 'source', 'replies', 'label'],
    num_rows: 1895
})

In [ ]:
dataset_dev = Dataset.from_pandas(data_dev)
dataset_dev

Dataset({
    features: ['Unnamed: 0', 'source', 'replies', 'label'],
    num_rows: 632
})

In [ ]:
total_train = pd.concat([data_dev,data_train], axis=0)
dataset_total_train = Dataset.from_pandas(total_train)

## Transformer Dataset to TensorFlow TF Dataset

Makin use of Transformer Dataset

In [ ]:
## Set up Tokenizer


tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, normalization=True)

## Tokenize with two sentences separated by [SEP]m, use source and reply as two sentences
def tokenize_function(dataset):
    return tokenizer(dataset["source"], dataset["replies"], padding="max_length", truncation=True)
    #return tokenizer(dataset["text"], padding=True, truncation=True)

# Set up Train data
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=True)
# Set up Dev Data
tokenized_dev_datasets = dataset_dev.map(tokenize_function, batched=True)


# Set up Total Data
tokenized_total_datasets = dataset_total_train.map(tokenize_function, batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/2 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

  0%|          | 0/1 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

  0%|          | 0/3 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
## Use data_collator to batch the dataset
data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
tf_train_dataset = tokenized_train_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

tf_validation_dataset = tokenized_dev_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)



tf_totaltrain_dataset = tokenized_total_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

## Tensor Flow Bert Model

Use Training Set to train and test against dev set

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# https://stackoverflow.com/questions/52041931/is-there-an-optimizer-in-keras-based-on-precision-or-recall-instead-of-loss
from keras import backend as K
THRESHOLD = 0.5
def precision(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))

    precision = tp / (tp + fp)
    return precision


def recall(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fn = K.sum(K.round(K.clip(y_true - y_pred_bin, 0, 1)))

    recall = tp / (tp + fn)
    return recall


def fbeta(y_true, y_pred, beta = 2, threshold_shift=0.5-THRESHOLD):   
    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall) 


In [ ]:


## Define Model
model = TFAutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2,
        hidden_dropout_prob=DROPOUT)

#model = TFAutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)


## Set up optimisation method, minimise which loss
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=tf.metrics.SparseCategoricalAccuracy(),
# )

# Use f1 score to capture information
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=[tf.metrics.SparseCategoricalAccuracy(), fbeta,precision,recall]
)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Fit model using Training ONLY
#model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=NUM_EPOCHS)


## Train using Train + Dev Set

In [ ]:
# Fit model
model.fit(tf_totaltrain_dataset, validation_data=tf_validation_dataset, epochs=NUM_EPOCHS)


Epoch 1/25
78/78 [==============================] - 96s 937ms/step - loss: 0.4145 - sparse_categorical_accuracy: 0.8049 - fbeta: 0.1671 - precision: 0.1494 - recall: 0.2084 - val_loss: 0.2515 - val_sparse_categorical_accuracy: 0.8908 - val_fbeta: 0.3014 - val_precision: 0.1946 - val_recall: 0.3667
Epoch 2/25
78/78 [==============================] - 71s 907ms/step - loss: 0.1949 - sparse_categorical_accuracy: 0.9287 - fbeta: 0.3194 - precision: 0.1918 - recall: 0.4034 - val_loss: 0.0680 - val_sparse_categorical_accuracy: 0.9794 - val_fbeta: 0.3770 - val_precision: 0.2193 - val_recall: 0.4877
Epoch 3/25
78/78 [==============================] - 71s 913ms/step - loss: 0.1072 - sparse_categorical_accuracy: 0.9659 - fbeta: 0.3705 - precision: 0.2129 - recall: 0.4703 - val_loss: 0.0532 - val_sparse_categorical_accuracy: 0.9842 - val_fbeta: 0.3864 - val_precision: 0.2237 - val_recall: 0.5000
Epoch 4/25
78/78 [==============================] - 70s 903ms/step - loss: 0.0681 - sparse_categorical_

## Test on Test Set

In [ ]:
test_df = pd.read_csv("/content/drive/My Drive/dataset_test.csv")
test_df.fillna(" ", inplace=True)
test_dataset = Dataset.from_pandas(test_df)  # Convert to Transformer Dataset

In [ ]:
# Convert to Keras input for Bert model
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
tf_test_dataset = tokenized_test_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
y_pred = model.predict(tf_test_dataset)

In [ ]:
# generate the csv for prediction
def generate_csv(pred, csv_name):
    ids = pd.Index(range(len(pred)), name='Id')
    predictions = pd.DataFrame(pred, index=ids)
    predictions.columns = ['Predicted']
    predictions.to_csv(csv_name)

def model_output_to_label(model_output):
    """Conver the output class of a tensorflow model to label"""
    logit_df = pd.DataFrame(model_output.to_tuple()[0], columns = ["0","1"])
    ## Choose highest logit as the predicted class
    logit_df["label"] = logit_df.apply(lambda x: 0 if x["0"] > x["1"] else 1, axis=1)
    return logit_df["label"]

In [ ]:
labels = model_output_to_label(y_pred)
generate_csv(labels, "/content/drive/My Drive/vanillabertmodel.csv")

In [ ]:
labels.value_counts()

0    437
1    121
Name: label, dtype: int64

In [ ]:
# Save model
#tf.keras.models.save_model(model, "saved_model.hp5", save_format="h5")

# Load model

#custom_metric = pickle.load(open("/content/drive/My Drive/bert_metric.pickle", 'rb'))
#loaded_model = tf.keras.models.load_model("saved_model.hp5", custom_objects=custom_metric)

# Predict Covid Tweets Label



In [ ]:
covid_df = pd.read_csv("/content/drive/My Drive/covid_bert_data.csv")
covid_df.fillna(" ", inplace=True)
covid_dataset = Dataset.from_pandas(covid_df)

In [ ]:
## Use data_collator to batch the dataset

tokenized_covid_dataset = covid_dataset.map(tokenize_function, batched=True)
tf_covid_dataset = tokenized_covid_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=20,
)

  0%|          | 0/18 [00:00<?, ?ba/s]

Streaming output truncated to the last 5000 lines.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned

In [ ]:
y_pred_covid = model.predict(tf_covid_dataset)


In [ ]:

labels2 = model_output_to_label(y_pred_covid)
generate_csv(labels2, "/content/drive/My Drive/covid_labels.csv")

In [ ]:
labels2.value_counts()

0    13049
1     4409
Name: label, dtype: int64